In [27]:
from arcgis.gis import GIS
import arcgis.network as network
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.features import Feature, FeatureSet

import sys
sys.path.append('../../')
from utils import get_config

from IPython.display import HTML


In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

In [3]:
analysis_url = gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

In [4]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)
cf_layer

<ClosestFacilityLayer url:"https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World">

In [5]:
sba_addresses = ['550 West C St. suite 550, San Diego',
                 '880 National City Blvd, San Diego',
                 '530 B St. Suite 700, San Diego',
                 '7675 Dagget St. Suite 340, San Diego']

In [6]:
sba_feat_list = []

for address in sba_addresses:
    hit = geocoding.geocode(address, max_locations=1)[0]
    sba_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    sba_feat_list.append(sba_feat)

In [7]:
sba_fset = FeatureSet(features=sba_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [8]:
sba_fset.sdf

,Loc_name,Status,Score,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,...,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,OBJECTID,SHAPE
0,World,M,100,"550 W C St, Ste 550, San Diego, California, 92101","550 W C St, Ste 550, San Diego, CA, 92101, USA","550 W C St, Ste 550",Subaddress,,,"550 W C St, Ste 550, San Diego, California, 92101",...,32.716748,-117.167841,32.717346,-117.168841,-117.166841,32.716346,32.718346,,1,"{""x"": -117.1678410271033, ""y"": 32.716747514306..."
1,World,M,100,"880 National City Blvd, National City, Califor...","880 National City Blvd, National City, CA, 919...",880 National City Blvd,StreetAddress,,,"880 National City Blvd, National City, Califor...",...,32.675143,-117.106795,32.675143,-117.107795,-117.105795,32.674143,32.676143,,2,"{""x"": -117.10679481570502, ""y"": 32.67514250187..."
2,World,M,100,"530 B St, Ste 700, San Diego, California, 92101","530 B St, Ste 700, San Diego, CA, 92101, USA","530 B St, Ste 700",Subaddress,,,"530 B St, Ste 700, San Diego, California, 92101",...,32.717828,-117.15973,32.718124,-117.16073,-117.15873,32.717124,32.719124,,3,"{""x"": -117.15973002703848, ""y"": 32.71782752253..."
3,World,M,100,"7675 Dagget St, Ste 340, San Diego, California...","7675 Dagget St, Ste 340, San Diego, CA, 92111,...","7675 Dagget St, Ste 340",Subaddress,,,"7675 Dagget St, Ste 340, San Diego, California...",...,32.823538,-117.158369,32.823228,-117.159369,-117.157369,32.822228,32.824228,,4,"{""x"": -117.15837534384727, ""y"": 32.82353845090..."


In [9]:
map1 = gis.map('San Diego County',zoomlevel=8)
map1.basemap = 'arcgis-light-gray'
map1


MapView(layout=Layout(height='400px', width='100%'))

In [10]:
map1.draw(sba_fset, symbol={"type": "esriSMS","style": "esriSMSSquare", "color": "blue","size": 10,})

In [11]:
user_coords = '-117.01707999999996, 32.67422000000005,'

In [12]:
reverse_geocode = geocoding.reverse_geocode({"x": user_coords.split(',')[0], 
                                              "y": user_coords.split(',')[1]})

In [13]:
user_location_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [14]:
user_location_fset = FeatureSet([user_location_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})

In [15]:
map1.draw(user_location_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","color": "[76,115,0,255]","size": 10})


In [16]:
user_location_fset.sdf

,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,AddNum,Address,Block,Sector,...,Subregion,Region,RegionAbbr,Territory,Postal,PostalExt,CntryName,CountryCode,OBJECTID,SHAPE
0,"5338-5338 Bonita Rd, Bonita, California, 91902","5338-5338 Bonita Rd, Bonita, CA, 91902, USA",5338-5338 Bonita Rd,StreetAddress,,,5338,5338 Bonita Rd,,,...,San Diego County,California,CA,,91902,,United States,USA,1,"{""x"": -117.01706142557242, ""y"": 32.67416657418..."


In [17]:
reverse_geocode

{'address': {'Match_addr': '5338-5338 Bonita Rd, Bonita, California, 91902',
  'LongLabel': '5338-5338 Bonita Rd, Bonita, CA, 91902, USA',
  'ShortLabel': '5338-5338 Bonita Rd',
  'Addr_type': 'StreetAddress',
  'Type': '',
  'PlaceName': '',
  'AddNum': '5338',
  'Address': '5338 Bonita Rd',
  'Block': '',
  'Sector': '',
  'Neighborhood': 'Bonita',
  'District': '',
  'City': 'Bonita',
  'MetroArea': 'San Diego Metro Area',
  'Subregion': 'San Diego County',
  'Region': 'California',
  'RegionAbbr': 'CA',
  'Territory': '',
  'Postal': '91902',
  'PostalExt': '',
  'CntryName': 'United States',
  'CountryCode': 'USA',
  'OBJECTID': 1,
  'SHAPE': {'x': -117.01706142557242,
   'y': 32.674166574183324,
   'spatialReference': {'latestWkid': 4326},
   'type': 'point'}},
 'location': {'x': -117.01706142557242,
  'y': 32.674166574183324,
  'spatialReference': {'latestWkid': 4326},
  'type': 'point'}}

In [37]:
result = cf_layer.solve_closest_facility(incidents=user_location_fset,
                                        facilities=sba_fset,
                                        default_target_facility_count=4,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Miles','TravelTime'])
result.keys()


dict_keys(['requestID', 'routes', 'facilities', 'messages'])

In [43]:
# result['routes']['features']

In [19]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [20]:
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])

In [21]:
df1 = routes_fset.sdf
df1

,ObjectID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,Total_TravelTime,Total_Miles,Total_Kilometers,Shape_Length,SHAPE
0,1,2,1,5338 Bonita Rd - Location 2,1,1,1,11.793344,8.279131,13.324187,0.133011,"{""paths"": [[[-117.01708054799997, 32.674221549..."
1,2,3,2,5338 Bonita Rd - Location 3,1,1,1,18.237284,13.635431,21.944333,0.216505,"{""paths"": [[[-117.01708054799997, 32.674221549..."
2,3,1,3,5338 Bonita Rd - Location 1,1,1,1,21.199016,13.137797,21.143388,0.210413,"{""paths"": [[[-117.01708054799997, 32.674221549..."
3,4,4,4,5338 Bonita Rd - Location 4,1,2,1,21.465569,17.628884,28.371332,0.271582,"{""paths"": [[[-117.01708054799997, 32.674221549..."


In [22]:
allocation_line_symbol_4 = {'type': 'esriSLS', 'style': 'esriSLSSolid',
                                'color': [0,0,255,100], 'width': 6}
map1.draw(routes_fset, symbol = allocation_line_symbol_4)

In [23]:
df1['sba_address'] = sba_addresses
df1['user_location_address'] = [user_location_feature.attributes['Match_addr'] for i in range(len(sba_addresses))]



In [25]:
df1[['sba_address','user_location_address','Total_Miles','Total_TravelTime']]


,sba_address,user_location_address,Total_Miles,Total_TravelTime
0,"550 West C St. suite 550, San Diego","5338-5338 Bonita Rd, Bonita, California, 91902",8.279131,11.793344
1,"880 National City Blvd, San Diego","5338-5338 Bonita Rd, Bonita, California, 91902",13.635431,18.237284
2,"530 B St. Suite 700, San Diego","5338-5338 Bonita Rd, Bonita, California, 91902",13.137797,21.199016
3,"7675 Dagget St. Suite 340, San Diego","5338-5338 Bonita Rd, Bonita, California, 91902",17.628884,21.465569


#### Try Geocoding address from registrant table

In [31]:
user_address = '581 Moss St, 91911, Chula Vista, CA'
matched_address = geocode(user_address, max_locations=1)[0]

In [32]:
#matched_address

In [35]:
user_address_feature = Feature(geometry=matched_address['location'], attributes=matched_address['attributes'])
user_address_feature

{"geometry": {"x": -117.08191738328053, "y": 32.61388094286144}, "attributes": {"Loc_name": "World", "Status": "M", "Score": 100, "Match_addr": "581 Moss St, Chula Vista, California, 91911", "LongLabel": "581 Moss St, Chula Vista, CA, 91911, USA", "ShortLabel": "581 Moss St", "Addr_type": "PointAddress", "Type": "", "PlaceName": "", "Place_addr": "581 Moss St, Chula Vista, California, 91911", "Phone": "", "URL": "", "Rank": 20, "AddBldg": "", "AddNum": "581", "AddNumFrom": "", "AddNumTo": "", "AddRange": "", "Side": "", "StPreDir": "", "StPreType": "", "StName": "Moss", "StType": "St", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "581 Moss St", "Block": "", "Sector": "", "Nbrhd": "", "District": "", "City": "Chula Vista", "MetroArea": "", "Subregion": "San Diego County", "Region": "California", "RegionAbbr": "CA", "Territory": "", "Zone": "", "Postal": "91911", "PostalExt": "1706", "Country": "US

In [45]:
feat_dict = matched_address['attributes']


In [46]:
feat_dict['org_name'] = 'San Diego and Imperial SBDC Network (Lead Office)'
feat_dict

{'Loc_name': 'World',
 'Status': 'M',
 'Score': 100,
 'Match_addr': '581 Moss St, Chula Vista, California, 91911',
 'LongLabel': '581 Moss St, Chula Vista, CA, 91911, USA',
 'ShortLabel': '581 Moss St',
 'Addr_type': 'PointAddress',
 'Type': '',
 'PlaceName': '',
 'Place_addr': '581 Moss St, Chula Vista, California, 91911',
 'Phone': '',
 'URL': '',
 'Rank': 20,
 'AddBldg': '',
 'AddNum': '581',
 'AddNumFrom': '',
 'AddNumTo': '',
 'AddRange': '',
 'Side': '',
 'StPreDir': '',
 'StPreType': '',
 'StName': 'Moss',
 'StType': 'St',
 'StDir': '',
 'BldgType': '',
 'BldgName': '',
 'LevelType': '',
 'LevelName': '',
 'UnitType': '',
 'UnitName': '',
 'SubAddr': '',
 'StAddr': '581 Moss St',
 'Block': '',
 'Sector': '',
 'Nbrhd': '',
 'District': '',
 'City': 'Chula Vista',
 'MetroArea': '',
 'Subregion': 'San Diego County',
 'Region': 'California',
 'RegionAbbr': 'CA',
 'Territory': '',
 'Zone': '',
 'Postal': '91911',
 'PostalExt': '1706',
 'Country': 'USA',
 'CntryName': 'United States',